In [1]:
import pandas as pd
import numpy as np
import csv
import tensorflow as tf
from matplotlib import pyplot as plt
from collections import Counter
from tqdm.auto import tqdm

2024-11-13 21:55:06.582790: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-13 21:55:06.585822: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-13 21:55:06.595931: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731531306.613897   33910 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731531306.618820   33910 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-13 21:55:06.635679: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
data = pd.read_csv("train.csv", header=None)
display(data)

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,0.1,e-399,NaN,w-219,NaN,NaN,w-77,w-71,w-72,w-109,...,w-196,w-185,NaN,NaN,NaN,w-215,1,0,0,0
1,0.3,e-399,NaN,w-219,NaN,NaN,w-77,w-71,w-72,w-109,...,w-196,w-185,NaN,NaN,NaN,w-215,1,0,0,0
2,0.6,e-397,NaN,w-215,NaN,NaN,w-77,w-73,w-72,w-109,...,w-197,w-185,NaN,NaN,NaN,w-213,1,0,0,0
3,0.8,e-396,NaN,w-214,NaN,NaN,w-79,w-72,w-72,w-109,...,w-196,w-185,NaN,NaN,NaN,w-213,1,0,0,0
4,1.0,e-394,NaN,w-211,NaN,NaN,w-81,w-70,w-72,w-109,...,w-196,w-185,NaN,NaN,NaN,w-211,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12163,-4.3,w-185,d-322,w-127,w-130,w-187,NaN,NaN,NaN,e-153,...,w-28,w-30,w-35,w-44,w-43,w-73,0,1,1,0
12164,-4.4,w-187,d-327,w-133,w-134,w-189,NaN,NaN,NaN,e-150,...,w-28,w-30,w-32,w-41,w-47,w-64,0,1,1,0
12165,-4.5,w-136,d-331,w-144,w-137,w-191,NaN,NaN,NaN,e-148,...,w-31,w-29,w-33,w-40,w-51,w-56,0,1,1,0
12166,-4.6,w-96,d-336,w-152,w-137,w-193,NaN,NaN,NaN,e-145,...,w-35,w-32,w-31,w-37,w-53,w-59,0,1,1,0


In [4]:
def parse_sensor(s, letters, max_val=501.0):
    if not s:
        return 1.0
    if s[0] not in letters:
        return 1.0
    return float(s[2:])/max_val

def parse_keys(keys):
    """
    up,down,left,right flags to one-hot encoded representation
    (up+left)   -> 0 (up)      -> 1 (up+right)   -> 2
    (left)      -> 3 (nothing) -> 4 (right)      -> 5
    (down+left) -> 6 (down)    -> 7 (down+right) -> 8
    """
    if np.array_equal(keys, [1,0,1,0]):
        return np.array([1.0,0.0,0.0, 0.0,0.0,0.0, 0.0,0.0,0.0])
    if np.array_equal(keys, [1,0,0,0]):
        return np.array([0.0,1.0,0.0, 0.0,0.0,0.0, 0.0,0.0,0.0])
    if np.array_equal(keys, [1,0,0,1]):
        return np.array([0.0,0.0,1.0, 0.0,0.0,0.0, 0.0,0.0,0.0])
    
    if np.array_equal(keys, [0,0,1,0]):
        return np.array([0.0,0.0,0.0, 1.0,0.0,0.0, 0.0,0.0,0.0])
    if np.array_equal(keys, [0,0,0,0]) or np.array_equal(keys, [0,0,1,1]):
        return np.array([0.0,0.0,0.0, 0.0,1.0,0.0, 0.0,0.0,0.0])
    if np.array_equal(keys, [0,0,0,1]):
        return np.array([0.0,0.0,0.0, 0.0,0.0,1.0, 0.0,0.0,0.0])

    if np.array_equal(keys, [0,1,1,0]):
        return np.array([0.0,0.0,0.0, 0.0,0.0,0.0, 1.0,0.0,0.0])
    if np.array_equal(keys, [0,1,0,0]):
        return np.array([0.0,0.0,0.0, 0.0,0.0,0.0, 0.0,1.0,0.0])
    if np.array_equal(keys, [0,1,0,1]):
        return np.array([0.0,0.0,0.0, 0.0,0.0,0.0, 0.0,1.0,0.0])

    raise ValueError(f"Unknown key combination: {keys}")
    

X = []
Y = []
with open("train.csv") as f:
    for row in tqdm(csv.reader(f)):
        velocity = np.array([float(row[0])])
        sensors = row[1:25]
        keys = np.fromiter((float(k) for k in row[25:]), dtype=float)
        keys = parse_keys(keys)
        walls = np.fromiter((parse_sensor(s, ("w",)) for s in sensors), dtype=float)
        enemy = np.fromiter((parse_sensor(s, ("e",)) for s in sensors), dtype=float)
        diamonds = np.fromiter((parse_sensor(s, ("d",)) for s in sensors), dtype=float)
        X.append(np.concatenate((velocity, walls, enemy, diamonds)))
        Y.append(keys)

X = np.array(X)
Y = np.array(Y)
X.shape, Y.shape

0it [00:00, ?it/s]

((12168, 73), (12168, 9))

In [5]:
def create_ff_model():
    input = tf.keras.Input(shape=(X.shape[1],), name="input")
    hidden = tf.keras.layers.Dense(64, activation="tanh", name="hidden1")(input)
    output = tf.keras.layers.Dense(Y.shape[1], activation='softmax', name='output')(hidden)

    model = tf.keras.Model(name="ff_model", inputs=input, outputs=output)
    model.compile(optimizer='adam', loss="categorical_crossentropy")
    return model

ff_model = create_ff_model()

2024-11-13 21:55:21.513454: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [6]:
ff_model.summary()

Model: "ff_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (InputLayer)              │ (None, 73)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ hidden1 (Dense)                 │ (None, 64)             │         4,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 9)              │           585 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,321 (20.79 KB)

 Trainable params: 5,321 (20.79 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
ff_model.fit(X, Y, epochs=20, validation_split=0.2)

Epoch 1/20
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.8270 - val_loss: 1.9112
Epoch 2/20
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 937us/step - loss: 0.8118 - val_loss: 1.9454
Epoch 3/20
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 959us/step - loss: 0.8336 - val_loss: 1.8888
Epoch 4/20
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 915us/step - loss: 0.8138 - val_loss: 1.9224
Epoch 5/20
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 877us/step - loss: 0.7985 - val_loss: 1.9733
Epoch 6/20
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 832us/step - loss: 0.8007 - val_loss: 1.9215
Epoch 7/20
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 854us/step - loss: 0.7997 - val_loss: 1.8911
Epoch 8/20
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 868us/step - loss: 0.7997 - val_loss: 1.8841
Epoch 9/20
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 877us/step - loss: 0.7993 - val_loss: 1.9578
Epoch 10/20
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 856us/step - loss: 0.7968 - val_loss: 1.9393
Epoch 11/20
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 839us/step - loss: 0.7997 - val_loss: 1.9785
Epoch 12/20
305/305 ━━━━━━━━━━━━

In [9]:
ff_model.export("../models/ff-one-hot")

INFO:tensorflow:Assets written to: ../models/ff-one-hot/assets


INFO:tensorflow:Assets written to: ../models/ff-one-hot/assets


Saved artifact at '../models/ff-one-hot'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 73), dtype=tf.float32, name='input')
Output Type:
  TensorSpec(shape=(None, 9), dtype=tf.float32, name=None)
Captures:
  129289310730000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  129289310727312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  129289310731536: TensorSpec(shape=(), dtype=tf.resource, name=None)
  129289310731920: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [10]:
import random
random.seed(42)

test_input = np.array([[random.random() for _ in range(73)]])

In [11]:
[float(f) for f in ff_model(test_input)[0]]

[1.1869574336742517e-05,
 1.0080448191729374e-05,
 0.0013925260864198208,
 0.0025202177930623293,
 0.004392880480736494,
 0.37408646941185,
 0.06280802935361862,
 0.5547780394554138,
 1.286232786945618e-10]

In [12]:
!python -m tf2onnx.convert --saved-model ../models/ff-one-hot --output ../models/ff-one-hot.onnx --opset 13

2024-11-13 21:56:30.605497: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-13 21:56:30.608359: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-13 21:56:30.617757: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731531390.633884   37317 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731531390.638666   37317 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-13 21:56:30.653822: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [13]:
import onnxruntime

In [14]:
onnx_session = onnxruntime.InferenceSession("../models/ff-one-hot.onnx")

In [17]:
[float(f) for f in onnx_session.run(["output_0"], {"input": test_input.astype(np.float32)})[0][0]]

[1.186958616017364e-05,
 1.0080449101224076e-05,
 0.0013925262028351426,
 0.002520219422876835,
 0.004392880946397781,
 0.3740863502025604,
 0.06280800700187683,
 0.5547780990600586,
 1.286232925723496e-10]